# Import all required libraries

In [30]:

import os
import selenium
from selenium import webdriver
import time
from PIL import Image
import io
import requests
from webdriver_manager.chrome import ChromeDriverManager


In [31]:
os.chdir('E:/practice/Python/Web_Scrapping')

# Install Chrome Driver

In [32]:

opts=webdriver.ChromeOptions()
opts.headless=True

driver = webdriver.Chrome(ChromeDriverManager().install(),options=opts)


[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\HP\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


# Search URL

In [33]:

search_url = "https://www.google.com/search?q={q}&tbm=isch&tbs=sur%3Afc&hl=en&ved=0CAIQpwVqFwoTCKCa1c6s4-oCFQAAAAAdAAAAABAC&biw=1251&bih=568"
driver.get(search_url.format(q='Car'))


# Write a function to take cursor to end of the page

In [34]:
def scroll_to_end(driver):
    driver.execute_script("window,scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)#sleep between interactions

# Write a function to get url of each image

In [35]:
#no licesnse issues

def getImageUrls(name,totalImgs,driver):
    
    search_url = "https://www.google.com/search?q={q}&tbm=isch&tbs=sur%3Afc&hl=en&ved=0CAIQpwVqFwoTCKCa1c6s4-oCFQAAAAAdAAAAABAC&biw=1251&bih=568"
    driver.get(search_url.format(q=name))

    img_urls = set()
    img_count = 0
    results_start = 0  
    
    while(img_count<totalImgs): #Extract actual images now
        
        scroll_to_end(driver)
        
        thumbnail_results = driver.find_elements_by_xpath("//img[contains(@class,'Q4LuWd')]")
        totalResults=len(thumbnail_results)
        print(f"Found: {totalResults} search results. Extracting links from {results_start}:{totalResults}")
        
        for img in thumbnail_results[results_start:totalResults]:
            
            img.click()
            time.sleep(2)
            actual_images = driver.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'https' in actual_image.get_attribute('src'):
                    img_urls.add(actual_image.get_attribute('src'))
            
            img_count=len(img_urls)
            
            if img_count >= totalImgs:
                print(f"Found: {img_count} image links")
                break
            else:
                print("Found:", img_count, "looking for more image links ...")                
                load_more_button = driver.find_element_by_css_selector(".mye4qd")
                driver.execute_script("document.querySelector('.mye4qd').click();")
                results_start = len(thumbnail_results)
    return img_urls
        
            

# Write a function to download each Image

In [36]:

def downloadImages(folder_path,file_name,url):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - COULD NOT DOWNLOAD {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        
        file_path = os.path.join(folder_path, file_name)
        
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SAVED - {url} - AT: {file_path}")
    except Exception as e:
        print(f"ERROR - COULD NOT SAVE {url} - {e}")

# Write a function to save each Image in Destination directory


In [37]:

def saveInDestFolder(searchNames,destDir,totalImgs,driver):
    for name in list(searchNames):
        path=os.path.join(destDir,name)
        if not os.path.isdir(path):
            os.mkdir(path)
        print('Current Path',path)
        totalLinks=getImageUrls(name,totalImgs,driver)
        print('totalLinks',totalLinks)

        if totalLinks is None:
            print('images not found for :',name)
            continue
        else:
            for i, link in enumerate(totalLinks):
                file_name = f"{i:150}.jpg"
                downloadImages(path,file_name,link)
            
searchNames=['Car','horses'] 
destDir=f'./Dataset2/'
totalImgs=5

saveInDestFolder(searchNames,destDir,totalImgs,driver)

Current Path ./Dataset2/Car
Found: 100 search results. Extracting links from 0:100
Found: 0 looking for more image links ...
Found: 0 looking for more image links ...
Found: 0 looking for more image links ...
Found: 0 looking for more image links ...
Found: 1 looking for more image links ...
Found: 2 looking for more image links ...
Found: 3 looking for more image links ...
Found: 3 looking for more image links ...
Found: 3 looking for more image links ...
Found: 4 looking for more image links ...
Found: 5 image links
totalLinks {'https://freesvg.org/img/SmallCar-v4.png', 'https://cdn.pixabay.com/photo/2017/02/01/12/23/car-2030056_960_720.png', 'https://freesvg.org/img/carsymbol5a.png', 'https://c1.wallpaperflare.com/preview/950/761/136/car-concept-vehicle-auto-speed-transportation.jpg', 'https://upload.wikimedia.org/wikipedia/commons/thumb/f/f6/Car.svg/1280px-Car.svg.png'}
SAVED - https://freesvg.org/img/SmallCar-v4.png - AT: ./Dataset2/Car\                                            